# Decoder-only based GPT (language model)

Here we take a transformer block, the decoder in particular, and use it for the task of language modeling. In general, this is how GPTs are trained. We will do this on a much smaller scale.

We take everything we've already built and leverage it in the way Karpathy implements a character level LM here:

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
!cp drive/MyDrive/Transformers/models/transformer_blocks.py .
!cp drive/MyDrive/Transformers/models/modules.py .
!cp -r drive/MyDrive/Transformers/data/ .

In [45]:
!pip install tokenmonster

In [46]:
import torch
from torch import nn
import numpy as np
from torch.utils.data import random_split
import sys
sys.path.append("~/")
from transformer_blocks import Transformer
from torch.utils.data import Dataset, DataLoader
import tokenmonster

In [47]:
harry_potter_text = " "
for i in range(4):
    book_num = i+1
    with open(f'/content/data/hp{book_num}.txt', 'r', encoding='utf-8') as f:
        harry_potter_text += f.read()
print(len(harry_potter_text))

2652650


## Tokenization
Instead of character level, we're going to model this LM using a tokenizer. in particular, we're going to try to use OpenAI's tiktoken with the gpt2 50k tokenizer. This might end up being too large of a vocab size given compute constraints, but

In [48]:
vocab = tokenmonster.load("fiction-2048-consistent-v1")
tokens = vocab.tokenize("This is a test.")

In [49]:
tokens

array([138, 918, 108, 318, 202,  17], dtype=uint16)

In [50]:
token_example = vocab.tokenize("hello world test monster tokenizer")

In [51]:
token_example

array([ 37, 445, 174, 785, 318, 202, 465, 547, 321, 169, 181, 218,  62],
      dtype=uint16)

In [52]:
[vocab.decode([token]) for token in token_example]

['',
 ' hel',
 'lo',
 ' world',
 ' te',
 'st',
 ' mon',
 'ster',
 ' to',
 'ke',
 'ni',
 'ze',
 'r']

In [53]:
tokens = np.array(vocab.tokenize(harry_potter_text), dtype=np.float16)

In [54]:
dataset = torch.tensor(tokens, dtype=torch.long)
print(dataset.shape, dataset.dtype)

torch.Size([1050223]) torch.int64


In [55]:
train_size = int(len(dataset) * 0.8)
test_size = int(len(dataset) * 0.1)
val_size = len(dataset) - train_size - test_size

train_data, test_data, val_data = dataset[:train_size], dataset[train_size:train_size+test_size], dataset[train_size+test_size:]

train_block = torch.tensor([train_data[i] for i in range(100)])
train_list = train_block.tolist()
print(vocab.decode(train_list))

val_block = torch.tensor([val_data[i] for i in range(100)])
val_list = val_block.tolist()
print(vocab.decode(val_list))

test_block = torch.tensor([test_data[i] for i in range(100)])
test_list = test_block.tolist()
print(vocab.decode(test_list))

 Harry Potter and the Sorcerer's Stone


CHAPTER ONE

THE BOY WHO LIVED

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be
rry muttered.  "Listen, you'd better go and get someone -"
"Dumbledore!" gasped Mr. Crouch. He reached out and seized a handful of Harrys robes, dragging him closer, though his eyes were staring over Harry's head.  "I need... see
 Harry.
"Would Harry Potter like a cup of tea?" he squeaked loudly, over Winky's sobs.
"Er - yeah, okay," said Harry.
Instantly, about six house-elves came trotting up behind him, bearing a large silver tray laden with a te


In [56]:
print(f"train set size: {train_size}, test: {test_size}, val: {val_size}")

train set size: 840178, test: 105022, val: 105023


In [57]:
torch.manual_seed(10000)
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(data):
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])  # for any sequence x, the target y will be the next 8 tokens
    return x, y

xb, yb = get_batch(train_data)

print(xb.shape, yb.shape)

context = xb[0, :2]
target = yb[0,1]
print(f"when input is {context.tolist()} the target: {target}")

torch.Size([32, 8]) torch.Size([32, 8])
when input is [64, 210] the target: 53


In [58]:
class HPDataset(Dataset):
    def __init__(self, data, block_size):
        self.data = data
        self.block_size = block_size

    def __len__(self):
        # Return the total number of possible sequences
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # Fetch a single sequence x and its corresponding target y
        x = self.data[idx:idx + self.block_size]
        y = self.data[idx + 1:idx + self.block_size + 1]
        return x, y

block_size = 8
train_dataset, val_dataset, test_dataset = HPDataset(train_data, block_size), HPDataset(val_data, block_size), HPDataset(test_data, block_size)

batch_size = 32
train_loader, val_loader, test_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True), DataLoader(val_dataset, batch_size=batch_size, shuffle=True), DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [59]:
print(len(train_loader))
print(len(test_loader))
print(len(val_loader))

26256
3282
3282


In [60]:
class zeptoGPT(nn.Module):
    """
    zepto because it's a really small GPT
    """
    def __init__(self, d_k, d_model, d_v, d_ff, num_heads, num_layers, vocab_size, dropout=0.1) -> None:
        super().__init__()
        self.decoder_transformer = Transformer(d_k, d_model, d_v, d_ff, num_heads, num_layers, vocab_size=vocab_size, mask=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        out = self.decoder_transformer(x)
        return self.fc(out)

In [61]:
def compute_loss(y_target, y_pred, loss_function):
    B, T, C = y_pred.shape
    y_pred = y_pred.view(B*T, C)
    _, max_indices = torch.max(y_pred, dim=1)
    y_target_list = y_target.tolist()
    max_indices = max_indices.tolist()
    y_target = y_target.view(B*T)
    return loss_function(y_pred, y_target)

In [ ]:
def generate(model, prompt: str, n = 50):
  prompt_tokenized = vocab.encode(prompt)
  for i in range(n):

    prompt_tensor = torch.tensor(prompt, dtype=torch.)
    next_token = model(prompt)

In [62]:
def predict_next_token(model, block):
  with torch.no_grad():
    y_pred = model(block)
    token_probs = nn.functional.softmax(y_pred, dim=-1)
    _, max_idx = torch.max(token_probs)
  return max_idx

In [63]:
def train(model, train_loader, val_loader, loss_function, optim, epochs, device):
    losses = [] #group losses for loss visualization
    running_loss = 0.0
    val_losses = []
    for epoch in range(epochs):
        model.train()
        print("Epoch %d / %d" % (epoch+1, epochs))
        print("-"*10)

        for i, batch_data in enumerate(train_loader):
            x, y = batch_data
            x, y = x.to(device), y.to(device)
            y_pred = model(x)

            loss = compute_loss(y, y_pred, loss_function)
            optim.zero_grad()
            loss.backward()
            optim.step()
            running_loss += loss.item()
            losses.append(loss)

            if (i+1) % 1000 == 0:
                print("Step: {}, average training loss over last 1000 steps: {:.4f}".format(i+1, running_loss/1000))
                running_loss = 0.0

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            correct_pred = 0.0
            num_samples = 0
            for i, batch_data in enumerate(val_loader):
                (y, x) = batch_data
                y, x = y.to(device), x.to(device)
                y_pred = model(x)
                loss = compute_loss(y, y_pred, loss_function)
                _, predicted_labels = torch.max(y_pred, 1)
                num_samples+=predicted_labels.shape[0]
                val_loss += loss.item()

            val_losses.append(val_loss)
        print("Epoch: {}, validation loss: {:.4f}".format(epoch+1, val_loss/len(val_loader)))

    return losses, val_losses

In [64]:
LEARNING_RATE = 1e-3
NUM_EPOCHS = 1
DROPOUT = 0.2
D_MODEL = 512
NUM_HEADS = 8
D_K = int(D_MODEL / NUM_HEADS)
D_V = D_K
D_FF = D_MODEL * 4
NUM_LAYERS = 4
VOCAB_SIZE = vocab.vocab_size
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [65]:
model = zeptoGPT(D_K, D_MODEL, D_V, D_FF, num_heads=NUM_HEADS, num_layers=NUM_LAYERS, vocab_size=VOCAB_SIZE)
model = model.to(DEVICE)

In [66]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

In [67]:
DEVICE

device(type='cuda', index=0)

In [68]:
train_loss, val_loss = train(model, train_loader, val_loader, torch.nn.functional.cross_entropy, optimizer, NUM_EPOCHS, DEVICE)

Epoch 1 / 1
----------
Step: 1000, average training loss over last 1000 steps: 4.5961
Step: 2000, average training loss over last 1000 steps: 4.2461
Step: 3000, average training loss over last 1000 steps: 4.1655
Step: 4000, average training loss over last 1000 steps: 4.1116
Step: 5000, average training loss over last 1000 steps: 4.0639
Step: 6000, average training loss over last 1000 steps: 4.0293
Step: 7000, average training loss over last 1000 steps: 4.0151
Step: 8000, average training loss over last 1000 steps: 3.9905
Step: 9000, average training loss over last 1000 steps: 3.9744
Step: 10000, average training loss over last 1000 steps: 3.9454
Step: 11000, average training loss over last 1000 steps: 3.9342
Step: 12000, average training loss over last 1000 steps: 3.9155
Step: 13000, average training loss over last 1000 steps: 3.8929
Step: 14000, average training loss over last 1000 steps: 3.8944
Step: 15000, average training loss over last 1000 steps: 3.8736
Step: 16000, average train

In [69]:
text_sample = train_data

In [70]:
print(text_sample[0])
test_block = torch.tensor([text_sample[i] for i in range(8)])

tensor(36)


In [71]:
test_block

tensor([ 36, 264,  62, 196,  36, 301,  64, 382])

In [72]:
test_list = test_block.tolist()

In [73]:
vocab.decode(test_list)

' Harry Potter'

In [74]:
test_block = torch.tensor([text_sample[i] for i in range(100)])
test_list = test_block.tolist()
vocab.decode(test_list)

" Harry Potter and the Sorcerer's Stone\n\n\nCHAPTER ONE\n\nTHE BOY WHO LIVED\n\nMr. and Mrs. Dursley, of number four, Privet Drive, were proud to say\nthat they were perfectly normal, thank you very much. They were the last\npeople you'd expect to be"